In [ ]:
!pip install git+https://github.com/BindsNET/bindsnet.git

In [2]:
import os
from bindsnet.datasets import MNIST
from bindsnet.encoding import PoissonEncoder
from SNN_supervised import load_config, create_network, create_monitorings, train_network, save_network
from torchvision import transforms

In [ ]:
config = load_config("config.json")

In [3]:
config = {
    "n_neurons": 100,
    "n_train": 10000,
    "n_test": 2000,
    "exc": 22.5,
    "inh": 120,
    "theta_plus": 0.05,
    "time": 250,
    "dt": 1.0,
    "intensity": 32,
    "progress_interval": 10,
    "update_interval": 10,
    "dataset": "mnist",
    "n_classes": 10,
    "lr": [1e-10, 1e-3]
}

In [ ]:
network = create_network(config["n_neurons"], config["exc"], config["inh"], config["dt"], config["theta_plus"], config["lr"])
network, spikes = create_monitorings(network, config["time"])

In [ ]:
dataset = MNIST(
    PoissonEncoder(time=config["time"], dt=config["dt"]),
    None,
    root=os.path.join("..", "..", "data", "MNIST"),
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Lambda(lambda x: x * config["intensity"])]
    ),
)

In [ ]:
network = train_network(network, dataset, spikes, config["n_train"], config["update_interval"], config["n_classes"], config["time"], config["n_neurons"], plot=True)

In [ ]:
save_network(network, "test.pth")